# Search and Filter DataFrames in PySpark

Once we have created our Spark Session, read in the data we want to work with and done some basic validation, the next thing you'll want to do is start exploring your dataframe. There are several option in PySpark to do this, so we are going to start with the following in this lecture, and continue to dive deeper in the next several lectures. 

### Agenda:

 - Introduce PySparks SQL funtions library 
 - Select method
 - Order By
 - Like Operator (for searching a string)
 - Substring Search
 - Is In Operator
 - Starts with, Ends with
 - Slicing
 - Filtering
 - Collecting Results as Objects

Let's get started!

In [ ]:
# First let's create our PySpark instance
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("Select").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

## Read in the DataFrame for this Notebook

In [6]:
path = '/home/iron/Documents/1.Learning/2. PYSPARK_BASICS/Read Write Validate Datasets/datasets/'
fifa = spark.read.csv(path+'fifa19.csv',inferSchema=True,header=True)


link1 = fifa.select('Photo').collect()[0][0]
print(link1)
fifa.printSchema()

https://cdn.sofifa.org/players/4/19/158023.png
root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From:

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

In [ ]:
# Take a look at the first few lines
fifa.limit(4).toPandas()

In [ ]:
print(fifa.printSchema())

## Select
There are a variety of functions you can import from pyspark.sql.functions. Check out the documentation for the full list available:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
# Import the functions we will need:
from pyspark.sql.functions import *
# countDistinct,avg,stddev
# abs # Absolute value
# acos # inverse cosine of col, as if computed by java.lang.Math.acos()

Since this is a sql function, the calls are pretty intuitive....

In [7]:
fifa_country = fifa.select(['Nationality','Name','Age'])

In [10]:
fifa_country.printSchema()

root
 |-- Nationality: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



**Order By**

In [ ]:
# who is the youngest player in the dataset?
fifa.select(['Nationality','Name','Age']).orderBy("Age").show(5)

In [11]:
# Who is the oldest player?
fifa.select(['Nationality','Name','Age']).orderBy(fifa["Age"].desc()).show(5)

+-----------------+-------------+---+
|      Nationality|         Name|Age|
+-----------------+-------------+---+
|           Mexico|     O. Pérez| 45|
|          England|K. Pilkington| 44|
|Trinidad & Tobago|    T. Warner| 44|
|            Japan|  S. Narazaki| 42|
|         Paraguay|    J. Villar| 41|
+-----------------+-------------+---+
only showing top 5 rows



**Like**

In [14]:
# If we wanted to look for all players that had "Barcelona" in their club title 
# We could use the like operator
fifa.select("Name","Club").where(fifa.Club.like("%Barcelona%")).show(5, False)

+---------------+------------+
|Name           |Club        |
+---------------+------------+
|L. Messi       |FC Barcelona|
|L. Suárez      |FC Barcelona|
|M. ter Stegen  |FC Barcelona|
|Sergio Busquets|FC Barcelona|
|Coutinho       |FC Barcelona|
+---------------+------------+
only showing top 5 rows



**Substrings**

.substr(starting postion,length)

Use this if you want to return a particular portion within a string

In [15]:
# Select last 4 characters of the photo column to understand all file types used
# This says return 
fifa.select("Photo",fifa.Photo.substr(-4,4)).show(5,False)

+----------------------------------------------+-----------------------+
|Photo                                         |substring(Photo, -4, 4)|
+----------------------------------------------+-----------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|.png                   |
|https://cdn.sofifa.org/players/4/19/20801.png |.png                   |
|https://cdn.sofifa.org/players/4/19/190871.png|.png                   |
|https://cdn.sofifa.org/players/4/19/193080.png|.png                   |
|https://cdn.sofifa.org/players/4/19/192985.png|.png                   |
+----------------------------------------------+-----------------------+
only showing top 5 rows



In [16]:
# Or we could get the date that the string of numbers there
fifa.select("Photo",fifa.Photo.substr(32, 11)).show(5,False)

+----------------------------------------------+------------------------+
|Photo                                         |substring(Photo, 32, 11)|
+----------------------------------------------+------------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|4/19/158023             |
|https://cdn.sofifa.org/players/4/19/20801.png |4/19/20801.             |
|https://cdn.sofifa.org/players/4/19/190871.png|4/19/190871             |
|https://cdn.sofifa.org/players/4/19/193080.png|4/19/193080             |
|https://cdn.sofifa.org/players/4/19/192985.png|4/19/192985             |
+----------------------------------------------+------------------------+
only showing top 5 rows



**ISIN**

You can also use ISIN to search for a list of options within a column.

In [17]:
fifa[fifa.Club.isin("FC Barcelona","Juventus")].limit(4).toPandas()

22/01/16 21:50:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96,33,28,26,6,11,15,14,8,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95,28,31,23,7,11,15,14,11,€127.1M
2,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,85,62,45,38,27,25,31,33,37,€164M
3,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,84,23,20,20,5,4,4,5,8,€153.5M


**Starts with Ends with**

Search for a specific case - begins with "x" and ends with "x"

In [19]:
fifa.select("Name","Club").where(fifa.Name.startswith("L")) \
                                  .where(fifa.Name.endswith("i")).limit(4).toPandas()

,Name,Club
0,L. Messi,FC Barcelona
1,L. Bonucci,Juventus
2,L. Fabiański,West Ham United
3,L. Pellegrini,Roma


#### Slicing a Dataframe

In [21]:
# Starting
print('Starting row count:',fifa.count())
print('Starting column count:',len(fifa.columns))

# Slice rows
df2 = fifa.limit(300)
print('Sliced row count:',df2.count())

# Slice columns
cols_list = fifa.columns[0:5]
df3 = fifa.select(cols_list)
print('Sliced column count:',len(df3.columns))

Starting row count: 18207
Starting column count: 88
Sliced row count: 300
Sliced column count: 5


**Slicing Method**

pyspark.sql.functions.slice(x, start, length)[source] <br>
Returns an array containing all the elements in x from index start (or starting from the end if start is negative) with the specified length.  <br>
<br>
*Note: indexing starts at 1 here*

In [43]:
# This is within an array
from pyspark.sql.functions import slice

df = spark.createDataFrame( [([1, 2, 3],), ([4, 5,6],)], ['x'] ) 
df.printSchema()
df.show()
df.select(slice(df.x, 2, 2).alias("sliced")).show()

root
 |-- x: array (nullable = true)
 |    |-- element: long (containsNull = true)

+---------+
|        x|
+---------+
|[1, 2, 3]|
|[4, 5, 6]|
+---------+

+------+
|sliced|
+------+
|[2, 3]|
|[5, 6]|
+------+



If we want to simply slice our dataframe (ie. limit the number of rows or columns) we can do this...

## Filtering Data

A large part of working with DataFrames is the ability to quickly filter out data based on conditions. Spark DataFrames are built on top of the Spark SQL platform, which means that is you already know SQL, you can quickly and easily grab that data using SQL commands, or using the DataFram methods (which is what we focus on in this course).

In [56]:
print(fifa.schema["Overall"].dataType)
c = fifa.filter("Overall > 50").count()
print(c)
# fifa.filter("Overall>50").toPandas()

IntegerType
18015


In [57]:
# Using SQL with .select()
fifa.filter("Overall>50").select(['ID','Name','Nationality','Overall']).limit(4).toPandas()

,ID,Name,Nationality,Overall
0,158023,L. Messi,Argentina,94
1,20801,Cristiano Ronaldo,Portugal,94
2,190871,Neymar Jr,Brazil,92
3,193080,De Gea,Spain,91


**Try it yourself!**

Edit the line below to select only closing values above 800

In [58]:
# Try it yourself!
# Edit the line below to select only overall scores of LESS THAN 80
fifa.filter("Overall<80").select(['ID','Name','Nationality','Overall']).limit(4).toPandas()

,ID,Name,Nationality,Overall
0,244369,V. Tsygankov,Ukraine,79
1,239818,Rúben Dias,Portugal,79
2,236632,David Neres,Brazil,79
3,233419,Raphinha,Brazil,79


In [59]:
fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).show()

+-----------+-----------------+---+-------+
|Nationality|             Name|Age|Overall|
+-----------+-----------------+---+-------+
|  Argentina|         L. Messi| 31|     94|
|   Portugal|Cristiano Ronaldo| 33|     94|
|     Brazil|        Neymar Jr| 26|     92|
|    Belgium|     K. De Bruyne| 27|     91|
|    Belgium|        E. Hazard| 27|     91|
|    Croatia|        L. Modrić| 32|     91|
|    Uruguay|        L. Suárez| 31|     91|
|      Spain|     Sergio Ramos| 32|     91|
|      Spain|           De Gea| 27|     91|
|   Slovenia|         J. Oblak| 25|     90|
|    Uruguay|         D. Godín| 32|     90|
|    Germany|         T. Kroos| 28|     90|
|     Poland|   R. Lewandowski| 29|     90|
|      Spain|      David Silva| 32|     90|
|     France|         N. Kanté| 27|     89|
|  Argentina|        P. Dybala| 24|     89|
|    England|          H. Kane| 24|     89|
|    Belgium|      T. Courtois| 26|     89|
|     France|     A. Griezmann| 27|     89|
|      Spain|  Sergio Busquets| 

### Collecting Results as Objects

The last thing we need to cover is collecting results as objects. If we wanted to say print individual names from an output, we need to essentially remove the item from the dataframe into an object. Like this

In [60]:
# Collecting results as Python objects
# you need the ".collect()" call at the end to "collect" the results
result = fifa.select(['Nationality','Name','Age','Overall']).filter("Overall>70").orderBy(fifa["Overall"].desc()).collect()


In [61]:
# Note the nested structure returns a nested row object
type(result[0])

pyspark.sql.types.Row

If we want to call on these results it would look something like this...

*Think of it like a matrix, first number is the row number and the second is the column number*

In [65]:
print("Best Player Over 70: ",result[0][1])
print("Nationality of Best Player Over 70: ",result[0][0])
print("")
print("Worst Player Over 70: ",result[-1][1])
print("Nationality of Worst Player Over 70: ",result[-1][0])

Best Player Over 70:  L. Messi
Nationality of Best Player Over 70:  Argentina

Worst Player Over 70:  Zapater
Nationality of Worst Player Over 70:  Spain


Rows can also be called to turn into dictionaries if needed

In [66]:
row.asDict()

NameError: name 'row' is not defined

Or iterated over like this...

In [67]:
for item in result[0]:
    print(item)

Argentina
L. Messi
31
94


Check out this link for more info on other methods:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

### Great job! That's it!